In [35]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [0]:
import os
os.chdir("drive/My Drive")

In [0]:
os.chdir("Colab Notebooks")

In [0]:
!ls

 CNN.ipynb			      results.ipynb
 colour_feature.npy		      Results.ipynb
'Copy of Results.ipynb'		      rsnet_feature.csv
 feature_extraction.ipynb	      rsnet_feature_test.csv
 final.ipynb			      shape_feature.npy
 finaltest.csv			      Sift_feature.csv
 final_train_500.csv		      sift_feature_extraction.ipynb
 finaltrain.csv			      sift_fisher_test.npy
 Histogram_feature.csv		      sift_fisher_train.npy
 histogram_feature_extraction.ipynb   Sift_results.ipynb
 histogram_feature_test.h5	      Surf_feature.csv
 histogram_test.npy		      surf_feature_extraction.ipynb
 histogram_train.npy		      surf_fisher_test.npy
 inception.ipynb		      surf_fisher_train.npy
 Inceptionwieght.ipynb		      test_colour_feature.npy
 Lbp_feature.csv		      test_data
 lbp_feature_extraction.ipynb	      testing.csv
 lbp_feature.npy		      test_lbp_feature.npy
 lbp_test.npy			      test_shape_feature.npy
 lbp_train.npy			      train_500.csv
 mnet_feature.csv		      train.csv
 mnet_feature_test.csv		      t

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import cv2
import math
from glob import glob
import os
import gc
from skimage.io import imread
import keras

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras import backend as k
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Concatenate
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)



ModuleNotFoundError: No module named 'keras'

In [ ]:
train_df = pd.read_csv("final_train_500.csv")

In [ ]:
train_df.iloc[0]

In [2]:
img_path = 'train_whale_500/'

#get the first 5 whale images
images = [(whale_img, whale_label) for (whale_img, whale_label) in zip(train_df.Image[:5], train_df.Id[:5])]

fig, m_axs = plt.subplots(1, len(images), figsize = (20, 10))
#show the images and label them
for ii, c_ax in enumerate(m_axs):
    c_ax.imshow(imread(os.path.join(img_path,images[ii][0])))
    c_ax.set_title(images[ii][1])

NameError: name 'train_df' is not defined

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test=train_test_split(train_df, test_size=0.20)


ModuleNotFoundError: No module named 'sklearn'

In [4]:
datagen=image.ImageDataGenerator(rescale=1./255, 
                                 validation_split = 0.1,
                                 rotation_range=20,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 horizontal_flip=True)
            
test_datagen = image.ImageDataGenerator(rescale=1./255) 

NameError: name 'image' is not defined

In [5]:
train_generator=datagen.flow_from_dataframe(
    dataframe=train_df, 
    directory="train_whale_500/", 
    x_col="Image", 
    y_col="Id", 
    seed = 42,
    class_mode="categorical", 
    target_size=(224,224), 
    batch_size=16, 
    subset = "training")

#train_datagen.fit(train_generator)

validation_generator = datagen.flow_from_dataframe(
    dataframe=train_df, 
    directory="train_whale_500/", 
    x_col="Image", 
    y_col="Id", 
     seed = 42,
    class_mode="categorical", 
    target_size=(224,224), 
    batch_size=16, 
    subset = "validation")

# make sure shuffle is set to false, so the predictions are done on the same order
# as they appear on the directory. batch_size should be 1 to make the
# predictions image by image


test_generator = test_datagen.flow_from_dataframe(
    dataframe=X_test, 
    directory="train_whale_500/", 
    x_col="Image", 
    y_col="Id", 
    seed = 42,
    class_mode=None,
    target_size=(224,224), 
    batch_size=1)


# test_generator = test_datagen.flow_from_directory(directory="test_folder", 
#     seed = 42, class_mode=None, target_size=(224,224), batch_size=1, shuffle = False)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

NameError: name 'datagen' is not defined

In [6]:
model_vgg16_conv = VGG16(weights=None, include_top=False)

# model_vgg16_conv.summary()

NameError: name 'VGG16' is not defined

In [7]:
# print('number of trainable weights weights',len(model_vgg16_conv.trainable_weights))
# for layer in model_vgg16_conv.layers[:8]:
#      layer.trainable= False
# print('number of trainable weights weights',len(model_vgg16_conv.trainable_weights))
# model_vgg16_conv.summary()

In [8]:
input = Input(shape=(224,224,3),name = 'image_input')

NameError: name 'Input' is not defined

In [9]:
output_vgg16_conv = model_vgg16_conv(input)

x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(500, activation='softmax', name='predictions')(x)

vgg16_model = Model(input=input, output=x)
# print('number of trainable weights weights',len(vgg16_model.trainable_weights))
# model_vgg16_conv.trainable = False
# print('number of trainable weights weights',len(vgg16_model.trainable_weights))
vgg16_model.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='weights_without_weights.hdf5', 
                               verbose=2, save_best_only=True)

early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20)



NameError: name 'model_vgg16_conv' is not defined

In [10]:
history=vgg16_model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50, callbacks = [checkpointer, early_stopping])
gc.collect()

NameError: name 'vgg16_model' is not defined

In [11]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [12]:
vgg16_model.save('vgg16_model_without_weights.h5')

NameError: name 'vgg16_model' is not defined

In [13]:
vgg16_model.summary()

NameError: name 'vgg16_model' is not defined

In [14]:
test_generator.reset()
pred=vgg16_model.predict_generator(test_generator,
steps=1850,
verbose=1)

NameError: name 'test_generator' is not defined

In [15]:
predicted_class_indices=np.argmax(pred,axis=1)

NameError: name 'pred' is not defined

In [16]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

NameError: name 'train_generator' is not defined

In [17]:
predictions

NameError: name 'predictions' is not defined

In [18]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)


NameError: name 'VGG16' is not defined

In [19]:
input = Input(shape=(224,224,3),name = 'image_input')

NameError: name 'Input' is not defined

In [20]:
output_vgg16_conv = model_vgg16_conv(input)

x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(500, activation='softmax', name='predictions')(x)

vgg16_model1 = Model(input=input, output=x)
# print('number of trainable weights weights',len(vgg16_model.trainable_weights))
# model_vgg16_conv.trainable = False
# print('number of trainable weights weights',len(vgg16_model.trainable_weights))
vgg16_model1.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='weights_with_weights.hdf5', 
                               verbose=2, save_best_only=True)

early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20)

history=vgg16_model1.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50, callbacks = [checkpointer, early_stopping])
gc.collect()

NameError: name 'model_vgg16_conv' is not defined

In [21]:
test_generator.reset()
pred=vgg16_model1.predict_generator(test_generator,
steps=1850,
verbose=1)

NameError: name 'test_generator' is not defined

In [22]:
predicted_class_indices=np.argmax(pred,axis=1)

NameError: name 'pred' is not defined

In [23]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

NameError: name 'train_generator' is not defined

In [24]:
predictions

NameError: name 'predictions' is not defined

In [25]:
from keras.models import Sequential

ModuleNotFoundError: No module named 'keras'

In [26]:
mod = Sequential()

mod.add(Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0', input_shape = (224, 224, 3)))

mod.add(BatchNormalization(axis = 3, name = 'bn0'))
mod.add(Activation('relu'))

mod.add(MaxPooling2D((2, 2), name='max_pool'))
mod.add(Conv2D(64, (3, 3), strides = (1,1), name="conv1"))
mod.add(Activation('relu'))
mod.add(AveragePooling2D((3, 3), name='avg_pool'))

mod.add(Flatten())
mod.add(Dense(500, activation="relu", name='rl'))
mod.add(Dropout(0.8))
mod.add(Dense(500, activation='softmax', name='sm'))


NameError: name 'Sequential' is not defined

In [27]:
mod.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='nrml_cnn.hdf5', 
                               verbose=2, save_best_only=True)

early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20)

history=mod.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50, callbacks = [checkpointer, early_stopping])
gc.collect()

NameError: name 'mod' is not defined

In [28]:
test_generator.reset()
pred=mod.predict_generator(test_generator,
steps=1850,
verbose=1)

predicted_class_indices=np.argmax(pred,axis=1)


NameError: name 'test_generator' is not defined

In [29]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

NameError: name 'train_generator' is not defined

In [30]:
predictions
count=0
for i in range(1850):
  if predictions[i]==X_test.iloc[i]['Id']:
    count+=1

NameError: name 'predictions' is not defined

In [31]:
count

NameError: name 'count' is not defined

In [32]:
train_generator=datagen.flow_from_dataframe(
    dataframe=train_df, 
    directory="train_whale_500/", 
    x_col="Image", 
    y_col="Id", 
    seed = 42,
    class_mode="categorical", 
    target_size=(224,224), 
    batch_size=16, 
    subset = "training")

#train_datagen.fit(train_generator)

validation_generator = datagen.flow_from_dataframe(
    dataframe=train_df, 
    directory="train_whale_500/", 
    x_col="Image", 
    y_col="Id", 
     seed = 42,
    class_mode="categorical", 
    target_size=(224,224), 
    batch_size=1000, 
    subset = "validation")

# make sure shuffle is set to false, so the predictions are done on the same order
# as they appear on the directory. batch_size should be 1 to make the
# predictions image by image


test_generator = test_datagen.flow_from_dataframe(
    dataframe=X_test, 
    directory="train_whale_500/", 
    x_col="Image", 
    y_col="Id", 
    seed = 42,
    class_mode=None,
    target_size=(224,224), 
    batch_size=1)


# test_generator = test_datagen.flow_from_directory(directory="test_folder", 
#     seed = 42, class_mode=None, target_size=(224,224), batch_size=1, shuffle = False)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

NameError: name 'datagen' is not defined

In [33]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential # to create a cnn model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (5,5), padding = 'Same', activation = 'relu', input_shape = (224,224,3)))
model.add(Conv2D(filters = 16, kernel_size = (5,5), padding = 'Same', activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
model.add(Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2), strides=(2,2)))
model.add(Dropout(0.25))

# fully connected
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dense(500, activation = "softmax"))

ModuleNotFoundError: No module named 'keras'

In [34]:
model.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='nrml_cnn.hdf5', 
                               verbose=2, save_best_only=True)

early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20)

history=model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=100,
                    epochs=50, callbacks = [checkpointer, early_stopping])
gc.collect()

NameError: name 'model' is not defined